<a href="https://colab.research.google.com/github/econ-sid/cheatsheet/blob/main/jobscheatsheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Redesigning the Jobs Cheatsheet HTML Version

Setup

In [ ]:
# === Cell 1: Setup & Imports ================================================
# Installs (safe to re-run)
!pip -q install pandas pandas_datareader plotly

import os
import numpy as np
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import plotly
from plotly.subplots import make_subplots

# Optional: set your FRED API key (uncomment and paste your key)
# os.environ["FRED_API_KEY"] = "YOUR_FRED_KEY_HERE"

FRED_API_KEY = os.getenv("FRED_API_KEY", None)

# Helper: robust FRED fetch
from pandas_datareader import data as pdr

def fetch_fred(series_ids, start="2015-01-01", end=None, api_key=FRED_API_KEY):
    """
    Fetch multiple FRED series into a single DataFrame with each series as a column.
    """
    frames = []
    for sid in series_ids:
        df = pdr.DataReader(sid, "fred", start=start, end=end, api_key=api_key)
        df.columns = [sid]
        frames.append(df)
    out = pd.concat(frames, axis=1)
    return out

# Small helpers
def yoy_pct(s): return s.pct_change(12) * 100.0
def last_non_na(s): return s.dropna().iloc[-1]
print("✅ Setup complete. pandas={}, plotly={}".format(pd.__version__, plotly.__version__))

✅ Setup complete. pandas=2.2.2, plotly=5.24.1


Tickers Needed

In [ ]:
# === Cell 2A: Your exact FRED tickers =======================================
SERIES = [
    "PAYEMS",          # Nonfarm payrolls (level, SA)
    "UNRATE",          # U-3 unemployment rate (%)
    "U6RATE",          # U-6 unemployment rate (%)
    "CIVPART",         # Labor force participation (%)
    "LNS12300060",     # Prime-age (25–54) employment-population ratio (%)
    "CES0500000003",   # Average hourly earnings, total private ($)
    "CES9091000001",   # (sector in your list)
    "USCONS",          # Construction
    "MANEMP",          # Manufacturing
    "USINFO",          # Information
    "USFIRE",          # Financial activities
    "USPBS",           # Professional & business services
    "USEHS",           # Education & health services
    "USLAH",           # Leisure & hospitality
    "USSERV",          # Service-providing (used as “Other Services” proxy if desired)
    "CES9092000001",   # (sector in your list)
    "CES9093000001",   # (sector in your list)
]

df = fetch_fred(SERIES, start="2015-01-01")

print("✅ Downloaded. Columns:", list(df.columns))
df.tail(3)


✅ Downloaded. Columns: ['PAYEMS', 'UNRATE', 'U6RATE', 'CIVPART', 'LNS12300060', 'CES0500000003', 'CES9091000001', 'USCONS', 'MANEMP', 'USINFO', 'USFIRE', 'USPBS', 'USEHS', 'USLAH', 'USSERV', 'CES9092000001', 'CES9093000001']


,PAYEMS,UNRATE,U6RATE,CIVPART,LNS12300060,CES0500000003,CES9091000001,USCONS,MANEMP,USINFO,USFIRE,USPBS,USEHS,USLAH,USSERV,CES9092000001,CES9093000001
DATE,,,,,,,,,,,,,,,,,
2025-06-01,159439,4.1,7.7,62.3,80.7,36.31,2943,8303,12736,2938,9244,22563,27329,17016,6033,5511,15128
2025-07-01,159518,4.2,7.9,62.2,80.4,36.43,2933,8302,12734,2931,9253,22553,27406,17022,6038,5506,15145
2025-08-01,159540,4.3,8.1,62.3,80.7,36.53,2918,8295,12722,2926,9250,22536,27452,17050,6050,5493,15157


Derived Metrics for Payrolls and Sector Table

In [ ]:
# === Cell 3 (updated): Derived metrics using 1-month change for sectors =====

# Monthly change in total payrolls
df["ΔPAYEMS"] = df["PAYEMS"].diff()

# Moving averages (3m/6m/9m/12m)
for w in (3, 6, 9, 12):
    df[f"ΔPAYEMS_{w}ma"] = df["ΔPAYEMS"].rolling(w).mean()

# YoY % change in Average Hourly Earnings
df["AHE_YoY"] = df["CES0500000003"].pct_change(12) * 100

# Identify macro vs. sector tickers
macro_cols = {
    "PAYEMS","UNRATE","U6RATE","CIVPART","LNS12300060","CES0500000003",
    "ΔPAYEMS","ΔPAYEMS_3ma","ΔPAYEMS_6ma","ΔPAYEMS_9ma","ΔPAYEMS_12ma","AHE_YoY"
}
sector_cols = [c for c in df.columns if c not in macro_cols]

# Calculate each sector's 1-month Δ and 12-month Δ
rows = []
for col in sector_cols:
    sector_change_1m = df[col].diff().dropna()
    sector_change_12m = (df[col] - df[col].shift(12)).dropna()
    if not len(sector_change_1m):
        continue
    curr_1m = sector_change_1m.iloc[-1]   # most recent monthly change
    chg_12m = sector_change_12m.iloc[-1] if len(sector_change_12m) else np.nan
    rows.append({
        "SectorID": col,
        "Current_1mChange": curr_1m,
        "Change_12m": chg_12m
    })

sector_df = pd.DataFrame(rows)

# Compute shares relative to total PAYEMS monthly change
total_change_1m = df["ΔPAYEMS"].iloc[-1]
sector_df["Pct_of_Total_Current"] = 100 * sector_df["Current_1mChange"] / total_change_1m

# Sort by contribution
sector_df = sector_df.sort_values("Current_1mChange", ascending=False).reset_index(drop=True)

print("✅ Derived complete (1-month sector changes).")
sector_df.head(10)


✅ Derived complete (1-month sector changes).


,SectorID,Current_1mChange,Change_12m,Pct_of_Total_Current
0,USEHS,46.0,862.0,209.090909
1,USLAH,28.0,232.0,127.272727
2,USSERV,12.0,75.0,54.545455
3,CES9093000001,12.0,184.0,54.545455
4,USFIRE,-3.0,80.0,-13.636364
5,USINFO,-5.0,-3.0,-22.727273
6,USCONS,-7.0,58.0,-31.818182
7,MANEMP,-12.0,-78.0,-54.545455
8,CES9092000001,-13.0,40.0,-59.090909
9,CES9091000001,-15.0,-86.0,-68.181818


Charts 1 - 3

In [ ]:
# === Cell 4: Charts 1–3 (ΔPAYEMS, UNRATE, U6RATE) ==========================
# keep only the last 3 years
df = df[df.index >= "2022-11-01"]
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=[
        "Monthly Change in Nonfarm Payrolls",
        "Official (U-3) Unemployment Rate",
        "Broad (U-6) Unemployment Rate"
    ]
)

# --- Chart 1: Job Gains with 3M MA ---
fig.add_trace(
    go.Bar(
        x=df.index, y=df["ΔPAYEMS"],
        name="Monthly Change",
        marker_color="#003865",
        opacity=0.8,
        showlegend=False
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=df.index, y=df["ΔPAYEMS_3ma"],
        mode="lines",
        line=dict(color="#D29F2A", width=3),
        showlegend=False
    ),
    row=1, col=1
)

# --- Add inline label with last value ---
last_date = df["ΔPAYEMS_3ma"].dropna().index[-1]
last_val = df["ΔPAYEMS_3ma"].dropna().iloc[-1]

# Format as thousands with rounding
val_text = f"{round(last_val):,}k"

fig.add_annotation(
    x=last_date, y=last_val,
    text=f"3m MA ({val_text})",
    xanchor="left", yanchor="middle",
    font=dict(size=12, color="#D29F2A"),
    showarrow=False,
    row=1, col=1
)

# --- Chart 2: U-3 ---
fig.add_trace(
    go.Scatter(
        x=df.index, y=df["UNRATE"],
        mode="lines",
        line=dict(color="#003865", width=3),
        showlegend=False
    ),
    row=1, col=2
)

# --- Chart 3: U-6 (same color as U-3, dotted) ---
fig.add_trace(
    go.Scatter(
        x=df.index, y=df["U6RATE"],
        mode="lines",
        line=dict(color="#003865", width=3),
        showlegend=False
    ),
    row=1, col=3
)

fig.update_layout(
    height=450, width=1150,
    template="plotly_white",
    margin=dict(l=40, r=20, t=40, b=40)
)

fig.show()


In [ ]:
# === Cell 5: Charts 4–6 (LFPR, Prime-Age E/Pop, AHE YoY) ===================
def _last_point(series):
    s = series.dropna()
    return (s.index[-1], s.iloc[-1]) if len(s) else (None, None)

line_color = "#003865"   # your blue
label_font = dict(size=12, color="#003865")

fig2 = make_subplots(
    rows=1, cols=3,
    subplot_titles=[
        "Labor Force Participation Rate",
        "Prime-Age (25–54) Employment-Population Ratio",
        "Average Hourly Earnings (YoY %)"
    ]
)

# --- Chart 4: LFPR ---
fig2.add_trace(
    go.Scatter(
        x=df.index, y=df["CIVPART"],
        mode="lines",
        line=dict(color=line_color, width=3),
        showlegend=False
    ),
    row=1, col=1
)
x_last, y_last = _last_point(df["CIVPART"])
if x_last is not None:
    fig2.add_annotation(
        x=x_last, y=y_last,
        text=f"{y_last:.1f}%",
        xanchor="left", yanchor="middle",
        font=label_font,
        showarrow=False,
        row=1, col=1
    )

# --- Chart 5: Prime-age E/Pop ---
fig2.add_trace(
    go.Scatter(
        x=df.index, y=df["LNS12300060"],
        mode="lines",
        line=dict(color=line_color, width=3),
        showlegend=False
    ),
    row=1, col=2
)
x_last, y_last = _last_point(df["LNS12300060"])
if x_last is not None:
    fig2.add_annotation(
        x=x_last, y=y_last,
        text=f"{y_last:.1f}%",
        xanchor="left", yanchor="middle",
        font=label_font,
        showarrow=False,
        row=1, col=2
    )

# --- Chart 6: AHE YoY ---
fig2.add_trace(
    go.Scatter(
        x=df.index, y=df["AHE_YoY"],
        mode="lines",
        line=dict(color=line_color, width=3),
        showlegend=False
    ),
    row=1, col=3
)
x_last, y_last = _last_point(df["AHE_YoY"])
if x_last is not None:
    fig2.add_annotation(
        x=x_last, y=y_last,
        text=f"{y_last:.1f}%",
        xanchor="left", yanchor="middle",
        font=label_font,
        showarrow=False,
        row=1, col=3
    )

fig2.update_layout(
    height=450, width=1150,
    template="plotly_white",
    margin=dict(l=40, r=20, t=40, b=40)
)

fig2.show()


In [ ]:
# === Cell 6: Moving-Average summary bars (3/6/9/12-month) ===================
def _fmt_k(x):
    try:
        return f"{round(x):,}k"
    except Exception:
        return ""

ma_points = {
    "3-Month":  df["ΔPAYEMS_3ma"].dropna().iloc[-1],
    "6-Month":  df["ΔPAYEMS_6ma"].dropna().iloc[-1],
    "9-Month":  df["ΔPAYEMS_9ma"].dropna().iloc[-1],
    "12-Month": df["ΔPAYEMS_12ma"].dropna().iloc[-1],
}

categories = list(ma_points.keys())
values = [ma_points[k] for k in categories]

fig_ma = go.Figure()

fig_ma.add_trace(
    go.Bar(
        x=values, y=categories,
        orientation="h",
        marker_color="#003865",
        opacity=0.9,
        showlegend=False
    )
)

# Inline labels on bars (e.g., "182k")
for cat, val in zip(categories, values):
    fig_ma.add_annotation(
        x=val, y=cat,
        text=_fmt_k(val),
        xanchor="left" if val >= 0 else "right",
        yanchor="middle",
        font=dict(size=12, color="#003865"),
        showarrow=False
    )

fig_ma.update_layout(
    xaxis_title="Jobs (thousands)",
    yaxis_title="",
    height=360, width=400,
    template="plotly_white",
    margin=dict(l=50, r=20, t=0, b=0)
)

fig_ma.show()


In [ ]:
# === Cell 7: Sector Breakdown Table (text color for negatives) ==============

def text_colors(series):
    """Return a list of text colors — red if negative, gray otherwise."""
    return ["#d9534f" if (not pd.isna(v) and v < 0) else "#333333" for v in series]

# Prepare data for display table
disp = sector_df.copy()
disp["Sector"] = disp["SectorID"].replace({
    "PAYEMS": "Total Nonfarm",
    "UNRATE": "U-3 Unemployment Rate",
    "U6RATE": "U-6 Unemployment Rate",
    "CIVPART": "Labor Force Participation",
    "LNS12300060": "Prime-Age E/Pop Ratio",
    "CES0500000003": "Average Hourly Earnings",
    "USCONS": "Construction",
    "MANEMP": "Manufacturing",
    "USINFO": "Information",
    "USFIRE": "Financial Activities",
    "USPBS": "Professional & Business Services",
    "USEHS": "Education & Health Services",
    "USLAH": "Leisure & Hospitality",
    "USSERV": "Other Services (proxy)", # Using USSERV as proxy
    "CES9091000001": "Goods Producing",
    "CES9092000001": "Private Service Providing",
    "CES9093000001": "Government"
})

# Calculate 12-month change as a percentage of total PAYEMS 12-month change
total_change_12m = (df["PAYEMS"].iloc[-1] - df["PAYEMS"].iloc[-13])
disp["Pct_of_Total_12m"] = 100 * disp["Change_12m"] / total_change_12m

# Format for display
disp["1M Change (thousands)"] = disp["Current_1mChange"].apply(lambda x: f"{x:,.0f}")
disp["% of Total 1M Change"] = disp["Pct_of_Total_Current"].apply(lambda x: f"{x:,.1f}%" if not pd.isna(x) else "")
disp["12M Change (thousands)"] = disp["Change_12m"].apply(lambda x: f"{x:,.0f}" if not pd.isna(x) else "")
disp["% of Total 12M Change"] = disp["Pct_of_Total_12m"].apply(lambda x: f"{x:,.1f}%" if not pd.isna(x) else "")


# Select and reorder columns for display
disp = disp[["Sector", "1M Change (thousands)", "% of Total 1M Change", "12M Change (thousands)", "% of Total 12M Change"]]


# Text color maps for numeric columns
txtcol_1m      = text_colors(sector_df["Current_1mChange"]) # Use original values for color mapping
txtcol_1m_pct  = text_colors(sector_df["Pct_of_Total_Current"]) # Use original values for color mapping
txtcol_12m     = text_colors(sector_df["Change_12m"]) # Use original values for color mapping
txtcol_12m_pct = text_colors(sector_df["Change_12m"]) # Use original values for color mapping - use Change_12m from sector_df


# Combine colors by column
cell_text_colors = [
    ["#333333"] * len(disp),   # Sector
    txtcol_1m,
    txtcol_1m_pct,
    txtcol_12m,
    txtcol_12m_pct
]

fig_table = go.Figure(
    data=[go.Table(
        header=dict(
            values=list(disp.columns),
            fill_color="#003865",
            font=dict(color="white", size=12),
            align="left"
        ),
        cells=dict(
            values=[disp[c] for c in disp.columns],
            font=dict(color=cell_text_colors, size=12),
            fill_color="#ffffff",
            align="left",
            height=25
        )
    )]
)

fig_table.update_layout(
    template="plotly_white",
    height=400,
    width=750,
    margin=dict(l=0, r=0, t=0, b=0)
)

fig_table.show()

In [ ]:
# === Cell 8: Combine all figs into one HTML page ============================
import plotly.io as pio
from datetime import datetime
from IPython.display import IFrame, HTML

page_title = "Jobs Day Cheat Sheet"
as_of = datetime.today().strftime("%B %Y")

# List all sections (table first)
sections = [
    ("Sector Contributions (Top Table)", fig_table),
    ("Headline Labor Indicators", fig),
    ("Labor Supply & Wage Growth", fig2),
    ("Moving Average Summary", fig_ma)
]

html_parts = []
loaded_plotlyjs = False

for title, f in sections:
    frag = pio.to_html(
        f,
        full_html=False,
        include_plotlyjs="cdn" if not loaded_plotlyjs else False,
        config={"displaylogo": False}
    )
    loaded_plotlyjs = True
    html_parts.append(f"""
    <section class="block">
      <h2 class="block-title">{title}</h2>
      {frag}
    </section>
    """)

# Arrange MA and table side-by-side (1st row layout)
# Extract table & MA fragments
frag_table = html_parts[0]
frag_headline = html_parts[1]
frag_labor = html_parts[2]
frag_ma = html_parts[3]

# Wrap table + MA horizontally
first_row = f"""
<section class="block side-by-side">
  <div class="half">{frag_table}</div>
  <div class="half">{frag_ma}</div>
</section>
"""

# Combine in desired order
body_html = first_row + frag_headline + frag_labor

page_html = f"""<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8" />
<title>{page_title}</title>
<style>
  :root {{
    --blue: #003865;
    --gold: #D29F2A;
    --ink: #222;
    --muted: #666;
  }}
  body {{
    margin: 0;
    padding: 24px 32px;
    background: #fff;
    color: var(--ink);
  }}
  header {{
    margin-bottom: 12px;
  }}
  h1 {{
    margin: 0;
    color: var(--blue);
    font-size: 26px;
    font-weight: 700;
  }}
  .sub {{
    margin: 4px 0 18px 0;
    color: var(--muted);
    font-size: 14px;
  }}
  .block {{
    margin: 20px 0 28px 0;
    padding-bottom: 8px;
    border-bottom: 1px solid #eee;
  }}
  .block-title {{
    margin: 0 0 8px 0;
    font-size: 18px;
    color: var(--blue);
    font-weight: 600;
  }}
  .side-by-side {{
    display: flex;
    flex-wrap: wrap;
    justify-content: space-between;
  }}
  .half {{
    flex: 1 1 48%;
  }}
  footer {{
    margin-top: 24px;
    padding-top: 12px;
    border-top: 1px solid #eee;
    font-size: 12px;
    color: var(--muted);
  }}
</style>
</head>
<body>
  <header>
    <h1>{page_title}</h1>
    <p class="sub">As of {as_of}. Sources: BLS via FRED. Payden Calculations.</p>
  </header>

  {body_html}

  <footer>
    <div>Notes: The data is updated via the Federal Reserve Economic Data (FRED) database maintained by St. Louis Fed.</div>
  </footer>
</body>
</html>"""

out_path = "jobs_cheatsheet.html"
with open(out_path, "w", encoding="utf-8") as f:
    f.write(page_html)

print(f"✅ Wrote {out_path}")



# Optional: to download in Colab
from google.colab import files
files.download(out_path)

# Inline display (works reliably in Colab)
HTML(page_html)

✅ Wrote jobs_cheatsheet.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>